In [2]:
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import botorch
from pak.choose import *
from pak.function import *
from pak.GPmodel import fit_gp_model
from pak.AugmentedEI import AugmentedExpectedImprovement
from skopt.sampler import Lhs
import warnings
warnings.filterwarnings("ignore")
torch.set_default_tensor_type(torch.DoubleTensor)

### EI

In [ ]:
# funcs = ['Fn1','Fn2','Fn7','Fn8','Fn10','Fn13','Fn18','Fn19','Fn22']
funcs = ['Fn1',]
noise = 'S'
for func in funcs:
    funcname = func + noise

    fn = choose(funcname)[0]
    f = choose(funcname)[1]
    search_space = [[-5.0, 5.0], [-5.0, 5.0]] 
    bounds = torch.tensor([[-5.0, -5.0], [5.0, 5.0]])
    budgets = 10

    for i in range(2):

        lhs = Lhs()
        initial_budgets = round(0.1*budgets)
        samples = lhs.generate(search_space, n_samples=initial_budgets)
        train_x = torch.tensor(samples)
        train_y = []
        for x in train_x:
            train_y.append(fn(x.numpy()))
        train_y = torch.tensor(train_y)

        obs_x = torch.tensor([])

        for j in tqdm(range(budgets-len(samples))):

            model, likelihood = fit_gp_model(train_x, train_y)

            policy = botorch.acquisition.analytic.ExpectedImprovement(
                model, best_f=train_y.min(), maximize = False
            )

            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

            next_x_obs = train_x[train_y.argmin()].reshape(1, 2)
            if j==0:
                obs_x = torch.tensor([next_x_obs[0].numpy().tolist()]*initial_budgets)

            next_y = torch.tensor(fn(next_x.numpy()))

            train_x = torch.cat([train_x, next_x])
            train_y = torch.cat([train_y, next_y])

            obs_x  = torch.cat([obs_x, next_x])

        obs_dict = {
            'Function':func,
            'Noise level':noise,
            'acquisition function':'EI',
            'Repeated round':int(i+1),
            'iteration':np.arange(1,budgets+1,1).astype(int),
            'x1':train_x[:,0].numpy(),
            'x2':train_x[:,1].numpy(),
            'Real value':f(train_x.numpy()),
            'obs_x1':obs_x[:,0].numpy(),
            'obs_x2':obs_x[:,1].numpy(),
            'Real value(obs)':f(obs_x.numpy()),
        }
        pd.DataFrame(obs_dict).to_csv('data\\obs.csv',mode='a',index=False,header=False)

        print(' ')
        print(funcname+': round'+str(i+1))

### EIM

In [ ]:
# funcs = ['Fn1','Fn2','Fn7','Fn8','Fn10','Fn13','Fn18','Fn19','Fn22']
funcs = ['Fn1',]
noise = 'S'
for func in funcs:
    funcname = func + noise

    fn = choose(funcname)[0]
    f = choose(funcname)[1]
    search_space = [[-5.0, 5.0], [-5.0, 5.0]] 
    bounds = torch.tensor([[-5.0, -5.0], [5.0, 5.0]])
    budgets = 10

    for i in range(1):

        lhs = Lhs()
        initial_budgets = round(0.1*budgets)
        samples = lhs.generate(search_space, n_samples=initial_budgets)
        train_x = torch.tensor(samples)
        train_y = []
        for x in train_x:
            train_y.append(fn(x.numpy()))
        train_y = torch.tensor(train_y)

        obs_M_x = torch.tensor([])
        total_x = torch.tensor([])

        for j in tqdm(range(budgets-len(samples))):

            model, likelihood = fit_gp_model(train_x, train_y)

            policy = botorch.acquisition.analytic.ExpectedImprovement(
                model, best_f=model(train_x).mean.min(), maximize = False
            )

            total = botorch.acquisition.analytic.UpperConfidenceBound(
                model, beta = 0.0, maximize = False
            )

            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

            next_x_total, acq_val_total = botorch.optim.optimize_acqf(
                total,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

            next_x_obs_M = train_x[model(train_x).mean.argmin()].reshape(1, 2)
            if j==0:
                obs_M_x = torch.tensor([next_x_obs_M[0].numpy().tolist()]*initial_budgets)
                total_x = torch.tensor([next_x_total[0].numpy().tolist()]*initial_budgets)

            next_y = torch.tensor(fn(next_x.numpy()))

            train_x = torch.cat([train_x, next_x])
            train_y = torch.cat([train_y, next_y])

            obs_M_x  = torch.cat([obs_M_x, next_x_obs_M])
            total_x = torch.cat([total_x, next_x_total])

        obs_M_dict = {
            'Function':func,
            'Noise level':noise,
            'acquisition function':'EIM',
            'Repeated round':i+1,
            'iteration':np.arange(1,budgets+1,1),
            'x1':train_x[:,0].numpy(),
            'x2':train_x[:,1].numpy(),
            'Real value':f(train_x.numpy()),
            'obs_M_x1':obs_M_x[:,0].numpy(),
            'obs_M_x2':obs_M_x[:,1].numpy(),
            'Real value(obs_M)':f(obs_M_x.numpy()),
        }
        pd.DataFrame(obs_M_dict).to_csv('data\\obs_M.csv',mode='a',index=False,header=True)

        total_dict = {
            'Function':func,
            'Noise level':noise,
            'acquisition function':'EIM',
            'Repeated round':i+1,
            'iteration':np.arange(1,budgets+1,1),
            'x1':train_x[:,0].numpy(),
            'x2':train_x[:,1].numpy(),
            'Real value':f(train_x.numpy()),
            'total_x1':total_x[:,0].numpy(),
            'total_x2':total_x[:,1].numpy(),
            'Real value(total)':f(total_x.numpy()),
        }
        pd.DataFrame(total_dict).to_csv('data\\total.csv',mode='a',index=False,header=True)
        print(' ')
        print(funcname+': round'+str(i+1))

### ALL

In [16]:
# funcs = ['Fn1','Fn2','Fn7','Fn8','Fn10','Fn13','Fn18','Fn19','Fn22']
funcs = ['Fn1',]
noise = 'S'
acq = 'AEI'
for func in funcs:
    funcname = func + noise

    fn = choose(funcname)[0]
    f = choose(funcname)[1]
    search_space = [[-5.0, 5.0], [-5.0, 5.0]] 
    bounds = torch.tensor([[-5.0, -5.0], [5.0, 5.0]])
    budgets = 300

    for i in range(1,30):

        lhs = Lhs()
        initial_budgets = round(0.1*budgets)
        samples = lhs.generate(search_space, n_samples=initial_budgets)
        train_x = torch.tensor(samples)
        train_y = []
        for x in train_x:
            train_y.append(fn(x.numpy()))
        train_y = torch.tensor(train_y)

        obs_x = torch.tensor([])
        obs_M_x = torch.tensor([])
        total_x = torch.tensor([])

        for j in tqdm(range(budgets-len(samples))):

            model, likelihood = fit_gp_model(train_x, train_y)

            policy = botorch.acquisition.analytic.ExpectedImprovement( #########
                model, best_f=model(train_x).mean.min(), maximize = False
            )

            total = botorch.acquisition.analytic.UpperConfidenceBound(
                model, beta = 0.0, maximize = False
            )

            next_x, acq_val = botorch.optim.optimize_acqf(
                policy,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

            next_x_total, acq_val_total = botorch.optim.optimize_acqf(
                total,
                bounds=bounds,
                q=1,
                num_restarts=40,
                raw_samples=100,
            )

            next_x_obs = train_x[train_y.argmin()].reshape(1, 2)
            next_x_obs_M = train_x[model(train_x).mean.argmin()].reshape(1, 2)

            if j==0:
                obs_x = torch.tensor([next_x_obs[0].numpy().tolist()]*initial_budgets)
                obs_M_x = torch.tensor([next_x_obs_M[0].numpy().tolist()]*initial_budgets)
                total_x = torch.tensor([next_x_total[0].numpy().tolist()]*initial_budgets)

            next_y = torch.tensor(fn(next_x.numpy()))

            train_x = torch.cat([train_x, next_x])
            train_y = torch.cat([train_y, next_y])

            obs_x  = torch.cat([obs_x, next_x])
            obs_M_x  = torch.cat([obs_M_x, next_x_obs_M])
            total_x = torch.cat([total_x, next_x_total])

        obs_dict = {
            'Function':func,
            'Noise level':noise,
            'acquisition function':acq,
            'Repeated round':int(i+1),
            'iteration':np.arange(1,budgets+1,1).astype(int),
            'x1':train_x[:,0].numpy(),
            'x2':train_x[:,1].numpy(),
            'Real value':f(train_x.numpy()),
            'obs_x1':obs_x[:,0].numpy(),
            'obs_x2':obs_x[:,1].numpy(),
            'Real value(obs)':f(obs_x.numpy()),
        }
        pd.DataFrame(obs_dict).to_csv('data\\obs.csv',mode='a',index=False,header=False)

        obs_M_dict = {
            'Function':func,
            'Noise level':noise,
            'acquisition function':acq,
            'Repeated round':int(i+1),
            'iteration':np.arange(1,budgets+1,1).astype(int),
            'x1':train_x[:,0].numpy(),
            'x2':train_x[:,1].numpy(),
            'Real value':f(train_x.numpy()),
            'obs_M_x1':obs_M_x[:,0].numpy(),
            'obs_M_x2':obs_M_x[:,1].numpy(),
            'Real value(obs_M)':f(obs_M_x.numpy()),
        }
        pd.DataFrame(obs_M_dict).to_csv('data\\obs_M.csv',mode='a',index=False,header=False)

        total_dict = {
            'Function':func,
            'Noise level':noise,
            'acquisition function':acq,
            'Repeated round':int(i+1),
            'iteration':np.arange(1,budgets+1,1).astype(int),
            'x1':train_x[:,0].numpy(),
            'x2':train_x[:,1].numpy(),
            'Real value':f(train_x.numpy()),
            'total_x1':total_x[:,0].numpy(),
            'total_x2':total_x[:,1].numpy(),
            'Real value(total)':f(total_x.numpy()),
        }
        pd.DataFrame(total_dict).to_csv('data\\total.csv',mode='a',index=False,header=False)
        print(' ')
        print(funcname+': round'+str(i+1))

  0%|          | 0/270 [00:00<?, ?it/s]

 
Fn1S: round2


  0%|          | 0/270 [00:00<?, ?it/s]

 
Fn1S: round3


  0%|          | 0/270 [00:00<?, ?it/s]

...

 
Fn1S: round28


  0%|          | 0/270 [00:00<?, ?it/s]

 
Fn1S: round29


  0%|          | 0/270 [00:00<?, ?it/s]

 
Fn1S: round30


In [17]:
import os
os.system("shutdown /s /t 60")

0